In [9]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

In [32]:
bn=gum.fastBN("A->B<-C;E->D<-C;E<-F->A")
bn.cpt("F").fillWith([0.6,0.4])
bn.cpt("C").fillWith([0.25,0.75])

bn.cpt("E")[:]=[[0.9,0.1],[0.1,0.9]]

bn.cpt("D")[{"E":0,"C":0}]=[0.99,0.01]
bn.cpt("D")[{"E":1,"C":0}]=[0.01,0.99]
bn.cpt("D")[{"E":0,"C":1}]=[0.99,0.01]
bn.cpt("D")[{"E":1,"C":1}]=[0.01,0.99]

bn.cpt("A").fillWith(bn.cpt("E"),"EF")
bn.cpt("B").fillWith(bn.cpt("D"),"DEC")

gnb.sideBySide(bn,bn.cpt("C"),bn.cpt("E"),bn.cpt("B"),bn.cpt("D"))

In [33]:
#gum.generateCSV(bn,"data.csv",5000)
generator=gum.BNDatabaseGenerator(bn)
generator.setRandomVarOrder()
generator.drawSamples(25000)
generator.toCSV("data.csv")

In [34]:
import pandas as pd
f=pd.read_csv("data.csv")
keep_col = ["A","B","D","E","F"]
new_f = f[keep_col]
new_f.to_csv("data2.csv", index=False)

In [35]:
learner=gum.BNLearner("data2.csv")
learner.useMIIC()
bn2=learner.learnBN()
gnb.sideBySide(bn,bn2)

G <!-- A --> A A <!-- B --> B B <!-- A->B --> A->B <!-- D --> D D <!-- E --> E E <!-- E->D --> E->D <!-- F --> F F <!-- F->A --> F->A <!-- F->E --> F->E


In [36]:
print("latent variables found between : ")
[(bn2.variable(x).name(),bn2.variable(y).name()) for x,y in learner.latentVariables()]

latent variables found between : 


[]

In [31]:
target="D"
evs="A"
ie=gum.LazyPropagation(bn)
ie2=gum.LazyPropagation(bn2)
gnb.sideBySide(ie.evidenceImpact(target,[evs]),ie2.evidenceImpact(target,[evs]))

In [21]:
import pyAgrum.causal as csl
import pyAgrum.causal.notebook as cslnb

modele=csl.CausalModel(bn)
modele2=csl.CausalModel(bn2)
cslnb.showCausalImpact(modele,target, {evs})
cslnb.showCausalImpact(modele2,target, {evs})

In [22]:
[(bn2.variable(i).name(),bn2.variable(j).name()) for (i,j) in learner.latentVariables()]

[('B', 'D')]

In [24]:
modele2=csl.CausalModel(bn2,[("X",("D","B"))])
cslnb.showCausalImpact(modele2,target, {evs})